# GitHub analysis client
This first version runs the analysis and takes for granted that pulsar already has the information it needs to do this.

In [ ]:
import pulsar
client = pulsar.Client('pulsar://localhost:6650')

In [ ]:
# Assumes it will receive messages ordered and in tuple format
def get_top_values(topic, num_values):
    
    consumer = client.subscribe(
                  topic=topic,
                  subscription_name=topic+'_sub')
    
    result_dict = {}
    for i in range(num_values):
        msg = consumer.receive()
        try:
            # Save the string message (decode from byte value)
            message = str(msg.value().decode())
            # Acknowledge that the message was received
            consumer.acknowledge(msg)
            # Evaluate the string (should have tuple format) and add to dict
            result_dict.update(dict([eval(message)]))
        except:
            consumer.negative_acknowledge(msg)
    
    return result_dict

## Test data and function before Pulsar implementation

In [ ]:
def get_top_values_test(num_values, message_list): 
    result_dict = {}
    for i in range(num_values):
        result_dict.update(dict([message_list[i]]))
    return result_dict

In [ ]:
top_languages_list = [('JavaScript', 414558), ('Java', 177839), ('Python', 267520), ('CSS', 57083), 
                ('PHP', 50818), ('Ruby', 22368), ('C++', 76267), ('C', 42708), ('Shell', 35257), 
                ('C#', 66798), ('Objective-C', 2120), ('R', 26508), ('VimL', 3812), ('Go', 24685), 
                ('Perl', 2498), ('CoffeeScript', 375), ('TeX', 5473), ('Swift', 14564), ('Scala', 3240), 
                ('Emacs Lisp', 35), ('Haskell', 2009), ('Lua', 5163), ('Clojure', 1278), ('Matlab', 11335), 
                ('Makefile', 7215), ('Groovy', 894), ('Puppet', 182), ('Rust', 11433), ('PowerShell', 4959)]

# 1. Top programming languages based on the number of projects developed

# 2. Repositories with most commits

# 3. Top languages with testdriven approach

# 4. Top languages with testdriven approach and continous integration